In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# データ読み込み
cars_x = pickle.load(open("train_feature.pkl", "br"))
cars_t = pickle.load(open("train_label.pkl", "br"))
x = np.array(cars_x, dtype=np.float32)
t = np.array(cars_t, dtype=np.int32)

t_one_hot = np.zeros([len(t), 4], dtype=np.float32)
t_one_hot[range(len(t)), t] = 1.

# t： 目標変数
#   訴求力（unacc 0, acc 1, good 2, vgood 3) 
# x： 入力変数
#   0: 購入時の価格
#   1: メンテナンス費用
#   2: ドア数
#   3: 乗車定員
#   4: トランクのサイズ
#   5: 安全性

In [ ]:
x[:10]

In [ ]:
t[:10]

In [ ]:
t_one_hot[:10]

In [ ]:
# softmax関数を用意
def softmax(ar):
    numerator = np.exp(ar)
    denominator = np.sum(np.exp(ar), axis=1, keepdims=True)
    
    return numerator / denominator

def outer_prod(a, b):
    return np.expand_dims(a, axis=2) * np.expand_dims(b, axis=1)

def report(i, cost, W, b):
    accuracy = evaluation(W, b)
    print("{:>6d}, cost: {:7.5f}, accuarcy: {:7.5f}".format(i, cost, accuracy))
    
def evaluation(W, b):
    a = activation(x, W, b)
    ok = np.sum(np.argmax(a, axis=1) == t)
    return ok / float(t.size)

In [ ]:
# モデルを用意
def activation(x, W, b):
    return np.dot(x, W) + b

In [ ]:
# パラメータの初期値設定
def parameter_init():
    W = np.array(
          [[ 0.40077151, -0.408296  , -0.37673183, -0.61619607],
           [ 0.40592896,  0.01773657,  0.12934329,  0.06548705],
           [ 0.10079057, -0.62975309,  0.86321079,  0.65757099],
           [-0.35792361, -0.92559288,  0.27577696,  0.27518349],
           [-0.4146183 ,  0.33060707,  0.91997293, -0.12877118],
           [ 0.75526573,  0.04088307, -0.82920987,  0.69621209]],
            dtype=np.float32)
    b = np.array([-0.20059004, -0.7472265 ,  0.6330216 , -0.57044723], dtype=np.float32)
    return W, b

W, b = parameter_init()

In [ ]:
# 現在の性能を評価
"{:5.3f}".format(evaluation(W, b))

In [ ]:
# コスト計算用の関数
def calc_cost(W, b):
    y = softmax(activation(x, W, b))
    prob = y[range(1200), t]
    minus_log_prob = -np.log(prob)
    return np.mean(minus_log_prob)

In [ ]:
# 実際の計算
W, b = parameter_init()
alpha = 0.1
history = list()

for i in range(5000):
    # 現状のコスト値を計算、記録
    # （calc_cost2が実装できていれば そちらへ切り替え）
    cost = calc_cost(W, b)
    history.append((cost))
    
    # 進捗モニタリング
    if i % 100 == 0:
        report(i, cost, W, b)

    # 実際の学習
    
    # 予測
    a = activation(x, W, b)
    y = softmax(a)
    
    # errorを出しておく
    e = y - t_one_hot

    # jを W、bで微分したものをそれぞれ求める
    j_W_deriv = outer_prod(x, e)
    j_b_deriv = e
    
    # 新しいパラメータ W、bを計算
    W_new = W - alpha * np.mean(j_W_deriv, axis=0)
    b_new = b - alpha * np.mean(j_b_deriv, axis=0)
    
    # 新旧パラメータをまとめて入れ替え
    W = W_new
    b = b_new
    
report(i, cost, W, b)

In [ ]:
# 学習曲線
plt.plot([t for t in history])
plt.xlabel("iterations")
plt.ylabel("cost")